# Build Networks from Articles Datasets

## Setup

In [1]:
import numpy as np
import pandas as pd
import networkx as nx

In [2]:
import matplotlib.pyplot as plt

from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats("svg")

In [3]:
import tqdm
import json

In [4]:
import tldextract

In [5]:
def url_to_domain(url):
    domain = tldextract.extract(url)
    domain = domain.domain + "." + domain.suffix
    return domain

In [6]:
def normalize_url(url):
    url = url.split("?")[0]
    url = url.split("://")[-1] if url[:4] == "http" else url
    if not url: return None
    url = url[:-1] if url[-1] == "/" else url
    return url

In [7]:
def parselist(s):
    s = s[2:-2]
    s = s.split("', '")
    return s

## Load Data

In [8]:
articles_data_1 = pd.read_csv("data/aylien_covid_articles_content_selected_1.csv")
articles_data_2 = pd.read_csv("data/aylien_covid_articles_content_selected_2.csv")
articles_data_3 = pd.read_csv("data/coaid_covid_articles_content.csv")

In [9]:
articles_data = pd.concat([articles_data_1, articles_data_2, articles_data_3])
articles_data = articles_data.drop_duplicates(subset=["url"])
articles_data = articles_data.reset_index(drop=True)

In [10]:
len(articles_data)

63813

In [11]:
domains = sorted(list(set(articles_data.domain.tolist())))
domains = domains
print(domains)
print(f"n domains: {len(domains)}")

['acs.org', 'androidcentral.com', 'aol.com', 'apa.org', 'apnews.com', 'apple.com', 'aptv.org', 'archdaily.com', 'archive.org', 'arizona.edu', 'avclub.com', 'azcentral.com', 'bgr.com', 'billboard.com', 'bizjournals.com', 'bloomberg.com', 'breitbart.com', 'brobible.com', 'buffalonews.com', 'businessinsider.com', 'bustle.com', 'buzzfeed.com', 'buzzorange.com', 'c-span.org', 'ca.gov', 'cbslocal.com', 'cbsnews.com', 'cbssports.com', 'cdc.gov', 'chicagotribune.com', 'chron.com', 'cisco.com', 'cnbc.com', 'cnet.com', 'cnn.com', 'complex.com', 'cool3c.com', 'cosmopolitan.com', 'cuny.edu', 'dailycaller.com', 'deadline.com', 'delta.com', 'denverpost.com', 'digitaltrends.com', 'ed.gov', 'eonline.com', 'epochtimes.com', 'ew.com', 'fastcompany.com', 'fivethirtyeight.com', 'forbes.com', 'foxbusiness.com', 'foxnews.com', 'go.com', 'google.com', 'gsmarena.com', 'harvard.edu', 'hbr.org', 'healthline.com', 'hindawi.com', 'huffingtonpost.com', 'informationisbeautiful.net', 'intel.com', 'investopedia.com',

## Check Data Issues

In [12]:
articles_per_domain = {d: 0 for d in domains}
for i, row in tqdm.tqdm(articles_data.iterrows(), total=len(articles_data)):
    articles_per_domain[row.domain] += 1

100%|██████████| 63813/63813 [00:04<00:00, 13338.52it/s]


In [13]:
nolink_ind = [i for i,l in enumerate(articles_data.links) if l == "[]"]
nolink_domains = articles_data.domain[nolink_ind].tolist()

In [14]:
proportion_nolink_bydomain = {d: (nolink_domains.count(d)/articles_per_domain[d]) if articles_per_domain[d] != 0 else 1.0 for d in domains}

In [15]:
baddomains = {k:v for k,v in proportion_nolink_bydomain.items() if v > 0.4}
baddomains_list = list(baddomains.keys())
baddomains

{'apa.org': 1.0,
 'bizjournals.com': 1.0,
 'bloomberg.com': 0.998109640831758,
 'cdc.gov': 0.46037099494097805,
 'hindawi.com': 1.0,
 'huffingtonpost.com': 1.0,
 'latimes.com': 1.0,
 'newsweek.com': 1.0,
 'nydailynews.com': 0.483974358974359,
 'princeton.edu': 1.0,
 'rev.com': 0.5,
 'texas.gov': 1.0,
 'thewheelerreport.com': 1.0,
 'weather.gov': 1.0}

## Network Generators

In [16]:
def generate_network_domainlevel_directlinks():
    links_by_domain_internal = {d1: {d2: 0 for d2 in domains} for d1 in domains}

    for i, row in tqdm.tqdm(articles_data.iterrows(), total=len(articles_data)):
        domain = row.domain

        links = parselist(row.links)
        links = [url_to_domain(url) for url in links]
        links = [l for l in links if l in domains]

        for l in links:
            links_by_domain_internal[domain][l] += 1

    links_by_domain_internal_normalized = {d: {k:(v/articles_per_domain[d]) if (articles_per_domain[d] != 0) else v for k,v in links_by_domain_internal[d].items()} for d in domains}
    
    G = nx.DiGraph()
    G.add_nodes_from(domains)
    
    for v1 in domains:
        outgoing = links_by_domain_internal[v1]
        for v2 in domains:
            w = outgoing[v2]
            if w > 0:
                G.add_edge(v1, v2, weight=w)

    return G

In [17]:
def generate_network_articlelevel_directlinks():
    G = nx.DiGraph()
    
    article_urls = [normalize_url(l) for l in articles_data.url.tolist()]
    article_domains = articles_data.domain.tolist()
    for i in range(len(article_urls)):
        G.add_node(article_urls[i], domain=article_domains[i])
    
    article_urls_set = set(article_urls)
    for i, row in tqdm.tqdm(articles_data.iterrows(), total=len(articles_data)):
        url = normalize_url(row.url)
        for l in parselist(row.links):
            l = normalize_url(l)
            if l is None: continue
            if l in article_urls_set:
                G.add_edge(url, l)

    return G

## Generated Networks

In [18]:
graph = generate_network_domainlevel_directlinks()
nx.draw(graph, node_size=30, width=0.2, with_labels=False, font_size=6)

100%|██████████| 63813/63813 [00:22<00:00, 2789.51it/s]


In [19]:
graph = generate_network_articlelevel_directlinks()
print(f"|V|={graph.number_of_nodes()}, |E|={graph.number_of_edges()}")

100%|██████████| 63813/63813 [00:07<00:00, 8497.66it/s]


|V|=63514, |E|=9789
